In [359]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import plotly.graph_objects as go
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc

To allow for flexible changes made some styling variables

In [360]:
WINDOW_WIDTH = 1920
WINDOW_HEIGHT = 927
BACKGROUND_COLOR = '#242424'
FONT_COLOR = '#FFFFFF'
FONT_FAMILY = 'Lato'
PADDING = '10px'
KPI_BOX_STYLE = {'borderRadius': '10px', 'backgroundColor': BACKGROUND_COLOR, 'padding': PADDING, 'color':FONT_COLOR, 'font-family':FONT_FAMILY, 'fontSize': '20px', 'justify':"center", 'align':"center"}

In [361]:
def generate_data(num_rows=1000):
    np.random.seed(0)
    years = np.random.choice(range(2000, 2024), num_rows)
    countries = np.random.choice(['Germany', 'France', 'Italy', 'Spain', 'UK', 'Belgium', 'Netherlands', 'Denmark', 'Sweden', 'Norway', 'Finland'], num_rows)
    construction_values = np.random.uniform(1000, 50000, num_rows)
    labor_wages = np.random.uniform(10, 50, num_rows)
    building_costs = np.random.uniform(500, 30000, num_rows)

    return pd.DataFrame({'Year': years, 'Country': countries,
                         'Construction Value': construction_values,
                         'Labor Wages': labor_wages,
                         'Building Cost': building_costs})

In [362]:
df = generate_data()

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.CYBORG])

In [363]:
# Define the app callbacks
@app.callback(
    Output('scatter-plot', 'figure'),
    Input('country-dropdown', 'value')
)
def update_scatter_plot(countries):
    df_filtered = df[df['Country'].isin(countries)]
    scatter_plot = go.Scatter(x=df_filtered['Year'],
                              y=df_filtered['Construction Value'],
                              mode='markers',
                              marker=dict(size=10, color='#1E88E5'))
    layout = go.Layout(title='Construction Value over Time',
                       xaxis=dict(title='Year'),
                       yaxis=dict(title='Construction Value'),
                       font=dict(family=FONT_FAMILY),
                       plot_bgcolor=BACKGROUND_COLOR,
                       paper_bgcolor=BACKGROUND_COLOR,
                       font_color=FONT_COLOR)
    return {'data': [scatter_plot], 'layout': layout}

In [364]:
@app.callback(
    Output('bar-chart', 'figure'),
    [Input('year-slider', 'value'), Input('top-10-check', 'value')]
)
def update_bar_chart(year, top10):
    df_filtered = df[df['Year'] == year]
    if 'TOP10' in top10:
        df_filtered = df_filtered.groupby('Country')['Construction Value'].mean().nlargest(10).reset_index()
    else:
        df_filtered = df_filtered.groupby('Country')['Construction Value'].mean().reset_index()

    bar_chart = go.Bar(x=df_filtered['Country'],
                       y=df_filtered['Construction Value'],
                       marker=dict(color='#FFC107'))
    layout = go.Layout(title=f'Construction Value by Country in {year}',
                       xaxis=dict(title='Country'),
                       yaxis=dict(title='Construction Value'),
                       font=dict(family=FONT_FAMILY),
                       plot_bgcolor=BACKGROUND_COLOR,
                       paper_bgcolor=BACKGROUND_COLOR,
                       font_color=FONT_COLOR)
    return {'data': [bar_chart], 'layout': layout}

In [365]:
@app.callback(
    Output('histogram', 'figure'),
    [Input('country-dropdown', 'value'), Input('wage-cost-radio', 'value')]
)
def update_histogram(countries, wage_or_cost):
    df_filtered = df[df['Country'].isin(countries)]
    histogram = go.Histogram(x=df_filtered[wage_or_cost],
                             nbinsx=20,
                             marker=dict(color='#00796B'))
    layout = go.Layout(title='Distribution of ' + wage_or_cost,
                       xaxis=dict(title=wage_or_cost),
                       yaxis=dict(title='Frequency'),
                       font=dict(family=FONT_FAMILY),
                       plot_bgcolor=BACKGROUND_COLOR,
                       paper_bgcolor=BACKGROUND_COLOR,
                       font_color=FONT_COLOR)
    return {'data': [histogram], 'layout': layout}

In [366]:
@app.callback(
    Output('pie-chart', 'figure'),
    Input('year-slider', 'value')
)
def update_pie_chart(year):
    df_filtered = df[df['Year'] == year]
    df_grouped = df_filtered.groupby('Country')['Construction Value'].mean()

    pie_chart = go.Pie(labels=df_grouped.index, values=df_grouped.values)
    layout = go.Layout(title=f'Average Construction Value by Country in {year}',
                       font=dict(family=FONT_FAMILY),
                       plot_bgcolor=BACKGROUND_COLOR,
                       paper_bgcolor=BACKGROUND_COLOR,
                       font_color=FONT_COLOR)
    return {'data': [pie_chart], 'layout': layout}

In [367]:
app.layout = html.Div(children=[
    html.H1("Construction Dashboard",
            style={'text-align':'center', 'padding':PADDING, 'color':FONT_COLOR, 'font-family':FONT_FAMILY}),

    dbc.Row([
        dbc.Col(html.H1('Avg Construction Value\n' + str(round(df['Construction Value'].mean(), 2)),
                        style=KPI_BOX_STYLE)),
        dbc.Col(html.H1('Avg Labor Wages\n' + str(round(df['Labor Wages'].mean(), 2)),
                        style=KPI_BOX_STYLE)),
        dbc.Col(html.H1('Avg Building Cost\n' + str(round(df['Building Cost'].mean(), 2)),
                        style=KPI_BOX_STYLE)),
        dbc.Col(html.H1('Number of Countries\n' + str(len(df['Country'].unique())),
                        style=KPI_BOX_STYLE))
    ], justify='center'),

    dbc.Row([
        dbc.Col([
            dcc.Dropdown(
                id='country-dropdown',
                options=[
                    {'label': country, 'value': country}
                    for country in df['Country'].unique()
                ],
                value=df['Country'].unique().tolist(),
                multi=True,
                placeholder='Select countries'
            ),

            dbc.Checklist(
                id='top-10-check',
                options=[{'label': 'Top 10 Countries Only', 'value': 'TOP10'}],
                value=[]
            ),

            dcc.Slider(
                id='year-slider',
                min=df['Year'].min(),
                max=df['Year'].max(),
                value=df['Year'].min(),
                marks={str(year): str(year) for year in df['Year'].unique()},
                step=None
            ),

            dcc.RadioItems(
                id='wage-cost-radio',
                options=[
                    {'label': 'Labor Wages', 'value': 'Labor Wages'},
                    {'label': 'Building Cost', 'value': 'Building Cost'},
                ],
                value='Labor Wages'
            ),
        ], md=3),

        dbc.Col([
            dcc.Graph(id='scatter-plot'),
            dcc.Graph(id='pie-chart'),
        ], md=3),

        dbc.Col([
            dcc.Graph(id='bar-chart'),
            dcc.Graph(id='histogram'),
        ], md=3),
    ]),
])

Finally run the app, it will run on your local network, e.g. http://127.0.0.1:XXXX/

In [ ]:
app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
127.0.0.1 - - [26/May/2023 16:04:45] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2023 16:04:45] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2023 16:04:45] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2023 16:04:45] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [26/May/2023 16:04:45] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 304 -
127.0.0.1 - - [26/May/2023 16:04:45] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [26/May/2023 16:04:45] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [26/May/2023 16:04:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2023 16:04:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2023 16:04:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2023 16:04:45] "POST /_dash-upd